# 2 Thread

In Chapter 1, we have seen how to use Process to accelerate your python program. In this chapter we will see:

1. What threads are
2. How to create threads and wait for them to finish
3. How to use a ThreadPoolExecutor
4. How to avoid race conditions
5. How to use the common tools that Python threading provide

## 2.1 What Is a Python Thread?

Generally speaking, a thread is a separate flow of execution, which means that your program will have many flow of execution happening at the same time. But for most Python 3 implementations **the threads do not actually execute at the same time**.

This is due to the python interpreter uses [GIL(Global Interpreter Lock)](https://realpython.com/python-gil/) to provide a thread safe memory management environment for the underlying C libraries. And **GIL limits only one Python thread can run at a time**.

## 2.2 When to use python thread?

As only one python thread can run at a time, if **tasks that spend much of their time waiting for external events are generally good candidates for threading**.

If **tasks that require heavy CPU computation and spend little time waiting for external events might not run faster at all after adding multi threading**.

**The above condition is true for code written in Python and running on the standard CPython implementation**. If your threads are written in C they have the ability to release the GIL and run concurrently. If you are running on a different Python interpreter implementation, check with the documentation to see how it handles threads.

**If you are running a standard Python implementation, writing in only Python, and have a CPU-bound problem, you should check out the multiprocessing module instead.**

So Why python still have thread, if they can't run parallely?
Thread can provide gains in design clarity for your program architecture. Most of the examples you’ll see in this chapter are not necessarily going to run faster because they use threads. Using threading in them helps to make the design cleaner and easier to reason about.

## 2.3 A simple thread

The Python standard library provides threading, which contains most of the primitives you’ll see in this article. Thread, in this module, nicely encapsulates threads, providing a clean interface to work with them.

To start a separate thread, you create a Thread instance and then tell it to .start():


In [1]:
import logging
import time
from threading import Thread

In [2]:
# define a thread task
def thread_task(name: str):
    logging.info(f"Thread {name} : start")
    time.sleep(2)
    logging.info(f"Thread {name} : stop")

In [5]:

## main process that launches thread
# set up logger

level="INFO"
date_format="%(asctime)s: %(message)s"
logging.basicConfig(level=level,datefmt=date_format)

# start the main process
logging.info("Main    : before creating thread")

# create thread
logging.info("Main    : before running thread-1")
thread1=Thread(target=thread_task,args=("thread-1",))
logging.info("Main    : before running thread-2")
thread2=Thread(target=thread_task,args=("thread-2",))

# start thread
thread1.start()
thread2.start()


logging.info("Main    : wait for the thread-1 to finish")
thread1.join()

logging.info("Main    : wait for the thread-2 to finish")
thread2.join()
# finish main process
logging.info("Main    : all done")



INFO:root:Main    : before creating thread
INFO:root:Main    : before running thread-1
INFO:root:Main    : before running thread-2
INFO:root:Thread thread-1 : start
INFO:root:Thread thread-2 : start
INFO:root:Main    : wait for the thread-1 to finish
INFO:root:Thread thread-1 : stop
INFO:root:Main    : wait for the thread-2 to finish
INFO:root:Thread thread-2 : stop
INFO:root:Main    : all done


In the above example, when you create a Thread, you pass it **a function and a list containing the arguments to that function**. In our case, you’re telling the Thread to run thread_task() and to pass it `thread-1` as an argument.

We recommend you to always give a custom thread name. There is **threading.get_ident()**, which returns a unique name for each thread, but these are usually neither short nor easily readable.


### The join() method

Try to comment the join(). You will notice the main finish before thread. Run src/SimpleThreadExample.py, it's more clear than the jupyter notebook.
So the join() method tells the main process to wait the thread that calls join().


### Daemon Threads

**In computer science, a daemon is a process that runs in the background.**

**In Python threading, a daemon thread will shut down immediately when the program exits.** As a result, you don't need to worry about shutting the daemon thread down.

If a program is running Threads that are not daemons, then the program will wait for those threads to complete before it terminates. Threads that are daemons, however, are just killed wherever they are when the program is exiting.

You can create a daemon thread by adding the **daemon=True** flag into the Thread constructor. Check src/DaemonThreadExample.py for example.

Run src/DaemonThreadExample.py, you will have below output

```text
14:24:03: Main    : before creating thread
14:24:03: Main    : before running thread-1
14:24:03: Main    : before running thread-2
14:24:03: Thread thread-1 : start
14:24:03: Thread thread-2 : start
14:24:03: Main    : wait for the thread-1 to finish
14:24:03: Main    : wait for the thread-2 to finish
14:24:03: Main    : all done
``` 

You can notice the two thread are started, by never finished. Because the main process finish before thread task, as the thread are daemon thread. They are automatically shut down after the main terminates. If you add the join, then the thread can finish normally.

## 2.4 Working with many threads

